In [4]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor



/Users/mrcremio/miniforge3/envs/data-science/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/mrcremio/miniforge3/envs/data-science/lib/python3.9/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefINS2_6SymIntEEENS2_8optionalINS2_10ScalarTypeEEENS6_INS2_6LayoutEEENS6_INS2_6DeviceEEENS6_IbEENS6_INS2_12MemoryFormatEEE
  Referenced from: <7968D834-00B6-3BC5-91B8-63E6BB9C4E4E> /Users/mrcremio/miniforge3/envs/data-science/lib/python3.9/site-packages/torchvision/image.so
  Expected in:     <98993E81-78F8-3504-938B-4D1F9450D4C6> /Users/mrcremio/miniforge3/envs/data-science/lib/python3.9/site-packages/torch/lib/libtorch_cpu.dylib
  warn(f"Failed to load image Python extension: {e}")


In [5]:

# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)


In [6]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [7]:
# Check that MPS is available
if not torch.backends.mps.is_available():
    if not torch.backends.mps.is_built():
        print("MPS not available because the current PyTorch install was not "
              "built with MPS enabled.")
    else:
        print("MPS not available because the current MacOS version is not 12.3+ "
              "and/or you do not have an MPS-enabled device on this machine.")

else:
    mps_device = torch.device("mps")

In [13]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
        
    def forward(self, x):
        x = self.flatten(x)
        x = self.linear_relu_stack(x)
        return x
        

model = NeuralNetwork().to(mps_device)
print(model)

        

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [14]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [15]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(mps_device), y.to(mps_device)
        
        # compute prediction error
        
        pred = model(X)
        loss = loss_fn(pred, y)
        
        #back propagation
        optimizer.zero_grad() #Sets the gradients of all optimized torch.Tensor s to zero.
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
        
        

In [16]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(mps_device), y.to(mps_device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= num_batches
    correct /= size
    
    print(f"Test Error: \n Acccuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [17]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.289531  [    0/60000]
loss: 2.286346  [ 6400/60000]
loss: 2.264397  [12800/60000]
loss: 2.268069  [19200/60000]
loss: 2.253819  [25600/60000]
loss: 2.212437  [32000/60000]
loss: 2.228852  [38400/60000]
loss: 2.186220  [44800/60000]
loss: 2.185386  [51200/60000]
loss: 2.159850  [57600/60000]
Test Error: 
 Acccuracy: 47.8%, Avg loss: 2.153153 

Epoch 2
-------------------------------
loss: 2.161308  [    0/60000]
loss: 2.155761  [ 6400/60000]
loss: 2.094396  [12800/60000]
loss: 2.115107  [19200/60000]
loss: 2.071895  [25600/60000]
loss: 1.999706  [32000/60000]
loss: 2.039511  [38400/60000]
loss: 1.951429  [44800/60000]
loss: 1.961720  [51200/60000]
loss: 1.894871  [57600/60000]
Test Error: 
 Acccuracy: 57.5%, Avg loss: 1.891542 

Epoch 3
-------------------------------
loss: 1.928611  [    0/60000]
loss: 1.896307  [ 6400/60000]
loss: 1.780783  [12800/60000]
loss: 1.825006  [19200/60000]
loss: 1.720826  [25600/60000]
loss: 1.666876  [32000/6